In [4]:
import os
import copy
import numpy as np
import librosa

from collections import namedtuple
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture

In [5]:
# (Label, Corresponding model).
LabeledModel = namedtuple('LabeledModel', ['label', 'model'])

# (Label, Path to files of that label).
LabeledPaths = namedtuple('LabeledPaths', ['label', 'paths'])

# (Label, scoring resut for given label).
LabeledResult = namedtuple('LabeledResult', ['label', 'score'])

In [6]:
def get_data_paths(paths_file: str, prefix: str='',
                   name_stop_symb: str='-') -> list[LabeledPaths]:
    """Construct a list of Examples (label, filepaths) from a given file
    containing filepaths separated by whitespaces.
    """
    
    # To be a list of LabeledPaths.
    training_set = []
    with open(paths_file) as file:
        
        # After first read get label and first path.
        while line := file.readline().rstrip():
                
            # Set part of the path before `name_stop_symb' as label.
            label = line.split(name_stop_symb)[0]
            ex = LabeledPaths(label, [os.path.join(prefix, line)])

            # Continue to read paths until there's no left.
            while line := file.readline().rstrip():
                ex.paths.append(os.path.join(prefix, line))

            training_set.append(ex)
    
    return training_set

In [14]:
def get_features(utterance: np.ndarray, sr: int, n_mfccs: int=20):
    """Get features from a given librosa-returned array. Returns n_mfccs * 3
    features: MFCC, their 1st and 2nd differences concatenated horizontally.
    """
    
    mfccs  = librosa.feature.mfcc(utterance, sr, n_mfcc=n_mfccs)
    mfccs1 = librosa.feature.delta(mfccs)
    mfccs2 = librosa.feature.delta(mfccs, order=2)
    
    # Librosa returns features in shape (n_mfcc, ...). We want mfccs to
    # be "columns" in our dataset => we transpose them.
    features = np.hstack((mfccs.T, mfccs1.T, mfccs2.T))
    return features

def get_features_path(filepaths: list[str], n_mfccs: int=20) -> np.ndarray:
    """Gets features from given files. Uses get_features for feature-extraction.
    """
    
    features = np.empty((0, n_mfccs * 3))
    for path in filepaths:
        utt, sr = librosa.load(path)
        utt = librosa.util.normalize(utt)
        features_utt = get_features(utt, sr, n_mfccs=n_mfccs)
        
        features = np.vstack((features, features_utt))
    
    return features

def map_adapt(ubm, X, max_iter=100, r=16):
    
    gmm = copy.deepcopy(ubm)
    
    for _ in range(max_iter):
        n = np.sum(gmm.predict_proba(X), axis=0).reshape(-1, 1) # (K, 1)
        X_tilde = (1 / n) * gmm.predict_proba(X).T.dot(X) # (K, F)
        alpha = (n / (n + r)).reshape(-1, 1) # (K, 1)
        gmm.means_ = alpha * X_tilde + (1 - alpha) * gmm.means_
    
    return gmm

In [15]:
def get_models(ubm: GaussianMixture, examples: list[LabeledPaths]) -> list[LabeledModel]:
    """Create and train models for a given set of LabeledPaths (label, filepaths)
    created via get_data_paths function. Returns labeled models of type
    (label, model). Labels are preserved from corresponding LabeledPaths.
    """
    
    # To be a list of labeled models.
    labeled_models = []
    for example in examples:
        
        # Get features from files.
        features = get_features_path(example.paths)
        
        # Create and train GMM using MAP-adaptation.
        gmm = map_adapt(ubm, features)
        
        # Add generated model to the list.
        lm = LabeledModel(example.label, gmm)
        labeled_models.append(lm)

    return labeled_models

def test_models(test_examples:  list[LabeledPaths],
                labeled_models: list[LabeledModel]) -> float:
    """Test models and return accuracy. Takes list of LabeledPaths (label, filepaths)
    and tests labeled models (label, model) against these examples.
    """
    
    # To be a list of results: Trues and Falses.
    results = []
    for test_example in test_examples:
        
        # Extract features for given speakers' filepaths.
        features = get_features_path(test_example.paths)
        
        # Get results for given speaker on all models in labeled_models.
        labeled_results = []
        for labeled_model in labeled_models:
            res = LabeledResult(labeled_model.label,
                                labeled_model.model.score(features))
            labeled_results.append(res)
        
        # Get index of a highest score and make corresponding label a prediction.
        scores_only = np.array([lr.score for lr in labeled_results], dtype=float)
        index = np.argmax(scores_only)
        label_pred = labeled_results[index].label
        
        # Check if prediction if correct and store result in results.
        results.append(label_pred == test_example.label)
    
    return sum(results) / len(results)

# Training

In [8]:
female_dir_ubm = './dataset_supervectors/train_data/UBM/female'
male_dir_ubm   = './dataset_supervectors/train_data/UBM/male'

In [9]:
features_ubm = np.empty((0, 60))
for filepath in os.listdir(female_dir_ubm):
    utt, sr = librosa.load(os.path.join(female_dir_ubm, filepath))
    utt = librosa.util.normalize(utt)
    features_utt = get_features(utt, sr)
    features_ubm = np.vstack((features_ubm, features_utt))

In [10]:
for filepath in os.listdir(male_dir_ubm):
    utt, sr = librosa.load(os.path.join(male_dir_ubm, filepath))
    utt = librosa.util.normalize(utt)
    features_utt = get_features(utt, sr)
    features_ubm = np.vstack((features_ubm, features_utt))

In [11]:
ubm = GaussianMixture(n_components=64,
                      covariance_type='diag',
                      n_init=3,
                      max_iter=200)

ubm.fit(features_ubm)

GaussianMixture(covariance_type='diag', max_iter=200, n_components=64, n_init=3)

## Train other models

In [16]:
train_dir = './dataset_vr'
train_paths_file = './train_data_paths.txt'

In [17]:
training_set = get_data_paths(train_paths_file, prefix=train_dir)

In [18]:
models = get_models(ubm, training_set)

# Testing

In [19]:
test_paths_file = './test_data_paths.txt'

In [20]:
testing_set = get_data_paths(test_paths_file, prefix=train_dir)

In [21]:
test_models(testing_set, models)

1.0